In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df1 = pd.read_csv('datasets\electricity_appliance_wise_data.csv')
df1['Date'] = pd.to_datetime(df1['Date'])
df1 = df1[df1['Date'].dt.year == 2021]
df1.reset_index(drop=True, inplace=True)
df1.head(2)

In [ ]:
def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [ ]:
df1['kap_zscore'] = zscore(df1['Kitchen Appliances'], 30)
df1['fridge_zscore'] = zscore(df1['Fridge'], 30)
df1['ac_zscore'] = zscore(df1['AC'], 30)
df1['oap_zscore'] = zscore(df1['Other Appliances'], 30)
df1['wm_zscore'] = zscore(df1['Washing Machine'], 3)

In [ ]:
df1[df1['kap_zscore'] > 5]

In [ ]:
df1[df1['fridge_zscore'] > 5]

In [ ]:
df1[df1['ac_zscore'] > 5]

In [ ]:
df1[df1['oap_zscore'] > 5]

In [ ]:
df1[df1['wm_zscore'] > 3]

In [ ]:
fig = go.Figure()
df_anoms = df1[df1['wm_zscore'] > 3]
fig.add_trace(go.Scatter(x=df1['Date'], y=df1['Washing Machine'],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=df_anoms['Date'], y=df_anoms['Washing Machine'],
                    mode='markers',
                    name='lines'))
fig.update_layout(title='Anomalies detected in Washing Machine')

In [ ]:
import smtplib, ssl
from smtplib import SMTPException
from pathlib import Path
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from email import encoders
import plotly
import datetime
import plotly.graph_objects as go
import plotly.offline as py

df_anoms_fridge = df1[df1['fridge_zscore'] > 3]
df_anoms_kap = df1[df1['kap_zscore'] > 3]
df_anoms_ac = df1[df1['ac_zscore'] > 3]
df_anoms_oap = df1[df1['oap_zscore'] > 3]
df_anoms_wm = df1[df1['wm_zscore'] > 3]

for i in df_anoms_wm['Date']:
    fig = go.Figure()
    df_anoms_temp = df_anoms_wm[df_anoms_wm['Date'] == i]
    df_anoms_temp.reset_index(drop=True, inplace=True)
    df_all_temp = df1[df1['Date'] <= i + datetime.timedelta(days=1)]
    df_all_temp.reset_index(drop=True, inplace=True)
    fig.add_trace(go.Scatter(x=df_all_temp['Date'], y=df_all_temp['Washing Machine'],
                        mode='lines',
                        name='Actual Consumption'))
    fig.add_trace(go.Scatter(x=df_anoms_temp['Date'], y=df_anoms_temp['Washing Machine'],
                        mode='markers',
                        name='High Fluctuations'))
    fig.update_traces(marker=dict(size=5, 
                                  line=dict(width=5,
                                            color='red')))
    i = str(i)
    i = i[:-9]
    plotly.io.write_image(fig, 'output_file'+i+'.pdf', format='pdf')
    SUBJECT = "Possible Detection of Faulty Device - Dated:" + i

    msg = MIMEMultipart()
    msg['Subject'] = SUBJECT 
    msg['From'] = 'debadriworkshop@gmail.com'
    msg['To'] = 'debadridtt@gmail.com'

    body = "Electra.AI has detected a possible faulty device. This can cause a power outage/trip/excessive electricity consumption"

    msg.attach(MIMEText(body, 'plain'))

    filename = 'output_file'+i+'.pdf'
    attachment = open('output_file'+i+'.pdf', "rb")

    part = MIMEBase('application', "octet-stream")
    part.set_payload((attachment).read())
    encoders.encode_base64(part)

    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)
    context = ssl.create_default_context()
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls(context=context)
    server.ehlo()
    server.login(msg['From'], '*******')
    server.sendmail(msg['From'], msg['To'], msg.as_string())
    print('Email sent successfully for Date:', i)